In [69]:
#setup credentials for BigQuery
from google.cloud import bigquery
credentials_path = './test-bigquery-ot-dcacff312913.json'
client = bigquery.Client.from_service_account_json(credentials_path)

In [70]:
#Imports
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [71]:
# Pull in the 1000 genomes pedigree information from a public BigQuery table
query = '''
WITH ranked_genes AS (
    SELECT locus2gene.study_id, 
        locus2gene.chrom, locus2gene.pos, locus2gene.ref, locus2gene.alt, 
        study_metadata.*, 
        genes.gene_name, locus2gene.y_proba_full_model,
        lead_snp_info.pval, lead_snp_info.beta, lead_snp_info.odds_ratio
    FROM `bigquery-public-data.open_targets_genetics.locus2gene` AS locus2gene
    INNER JOIN `bigquery-public-data.open_targets_genetics.studies` AS study_metadata
    ON locus2gene.study_id = study_metadata.study_id
    INNER JOIN `bigquery-public-data.open_targets_genetics.genes` AS genes
    ON locus2gene.gene_id = genes.gene_id
    INNER JOIN `bigquery-public-data.open_targets_genetics.disease_variant_gene` AS lead_snp_info
    ON locus2gene.study_id = lead_snp_info.study_id 
        AND locus2gene.chrom = lead_snp_info.lead_chrom
        AND locus2gene.pos = lead_snp_info.lead_pos
        AND locus2gene.ref = lead_snp_info.lead_ref
        AND locus2gene.alt = lead_snp_info.lead_alt
        #AND locus2gene.gene_id = lead_snp_info.gene_id
    WHERE locus2gene.study_id LIKE '%NEALE2%'
        #AND study_metadata.trait_reported LIKE '%%'
        AND locus2gene.study_id NOT LIKE '%raw%'
    )
SELECT * from ranked_genes
LIMIT 10
'''


# Run the query
query_job = client.query(query)

# Get the results as a pandas dataframe
l2g = query_job.to_dataframe()
l2g

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: b1835bb8-da5a-43c1-9d39-bd50f76304f1


In [65]:
l2g.sort_values("pval")

,study_id,chrom,pos,ref,alt,study_id_1,ancestry_initial,ancestry_replication,n_cases,n_initial,...,num_assoc_loci,source,trait_reported,trait_efos,trait_category,gene_name,y_proba_full_model,pval,beta,odds_ratio
0,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,ASPG,0.022855,4.118390e-08,NaN,3.675035
1,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,TMEM179,0.041173,4.118390e-08,NaN,3.675035
2,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,AKT1,0.015486,4.118390e-08,NaN,3.675035
3,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,SIVA1,0.013122,4.118390e-08,NaN,3.675035
4,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,INF2,0.024952,4.118390e-08,NaN,3.675035
5,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,ADSS1,0.017039,4.118390e-08,NaN,3.675035
6,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,TDRD9,0.040477,4.118390e-08,NaN,3.675035
7,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,RD3L,0.029009,4.118390e-08,NaN,3.675035
8,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,ATP5MJ,0.026702,4.118390e-08,NaN,3.675035
9,NEALE2_20002_1456,14,104356581,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,ZBTB42,0.018529,4.118390e-08,NaN,3.675035


Family_ID
1463     6
1350     6
1459     6
1456     6
1454     6
1451     6
1447     6
1444     6
1423     6
1418     6
1408     6
1377     6
1362     6
1420     6
1334     6
13291    6
1330     6
1340     6
1341     6
1346     6
Name: Individual_ID, dtype: int64